In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import write
from IPython.display import display, Audio
import scipy.integrate as intgr
import scipy.signal as sig
from numpy import linalg as LA
from mpl_toolkits.axes_grid1 import ImageGrid



In [3]:
import scipy.io
mat = scipy.io.loadmat('5_fingertip_14_6_E_nylon_horizontal_malena.mat') 

FileNotFoundError: [Errno 2] No such file or directory: '5_fingertip_14_6_E_nylon_horizontal_malena.mat'